# Keras入门课4：自编码器

In [1]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
import numpy as np

Using TensorFlow backend.


In [2]:

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
 
x_train, y_train = mnist.train.images, mnist.train.labels
x_test, y_test = mnist.test.images, mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


归一化，将图像的像素归到0~1

这样做的一个原因是 数字图像取值范围是0~255

In [4]:
x_train = x_train/255
x_test = x_test/255

In [5]:
# 参数
batch_size = 128
n_epoch = 20

# 图片维度
img_rows, img_cols = 28, 28
# denoising autoencoder参数
n_visible = img_rows * img_cols
n_hidden = 500
corruption_level = 0.3  # ?

In [6]:
# 添加噪声

X_train = x_train
X_test  = x_test

X_train_noisy = X_train + corruption_level * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + corruption_level * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
print(X_train_noisy.shape)
print(X_test_noisy.shape)


(55000, 784)
(10000, 784)


In [7]:

# 构建autoencoder 模型
input_img = Input(shape=(n_visible,))
encoded=Dense(n_hidden,activation='relu')(input_img)
decoded=Dense(n_visible,activation='sigmoid')(encoded)

autoencoder=Model(inputs=input_img,outputs=decoded)
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')
autoencoder.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_2 (Dense)              (None, 784)               392784    
Total params: 785,284
Trainable params: 785,284
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 训练
autoencoder.fit(X_train_noisy,X_train,epochs=n_epoch,batch_size=batch_size,
                shuffle=True,verbose=1,validation_data=(X_test_noisy,X_test))


Train on 55000 samples, validate on 10000 samples
Epoch 1/20
55000/55000 [==============================] - 16s 298us/step - loss: 0.0719 - val_loss: 0.0055
Epoch 2/20
55000/55000 [==============================] - 15s 278us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 3/20
55000/55000 [==============================] - 16s 300us/step - loss: 0.0044 - val_loss: 0.0044
Epoch 4/20
55000/55000 [==============================] - 15s 276us/step - loss: 0.0043 - val_loss: 0.0043
Epoch 5/20
55000/55000 [==============================] - 15s 264us/step - loss: 0.0042 - val_loss: 0.0042
Epoch 6/20
55000/55000 [==============================] - 15s 269us/step - loss: 0.0042 - val_loss: 0.0042
Epoch 7/20
55000/55000 [==============================] - 15s 273us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 8/20
55000/55000 [==============================] - 15s 279us/step - loss: 0.0041 - val_loss: 0.0041
Epoch 9/20
55000/55000 [==============================] - 19s 339us/step - loss: 0.0041 - val_

In [13]:
# 评价模型
evaluation=autoencoder.evaluate(X_test_noisy,X_test,batch_size=batch_size,verbose=1)
print('\nSummary: Loss over the test dataset: %.2f' % evaluation)
# Summary: Loss over the test dataset: 0.10

10000/10000 [==============================] - 1s 57us/step

Summary: Loss over the test dataset: 0.00


In [14]:
evaluation

0.004081567785888911